In [5]:
%load_ext autoreload
%autoreload 2

https://www.stata.com/features/overview/logistic-regression/

In [6]:
import numpy as np
import pandas as pd

/Users/fukuitomohiro/.pyenv/versions/3.7.3/envs/quantmetrics/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [7]:
import quantmetrics
from quantmetrics.datasets import webuse
from quantmetrics.discrete_choice import logistic

In [8]:
webuse('lbw')

In [9]:
data = pd.read_stata("data/lbw")

In [10]:
X = data.drop('low', axis=1)
y = data['low']
logistic(X, y)

   id  age  lwt  ptl  ht  ui  ftv   bwt  race_black  race_other  smoke_smoker
0  85   19  182    0   0   1    0  2523           1           0             0
1  86   33  155    0   0   0    3  2551           0           1             0
2  87   20  105    0   0   0    1  2557           0           0             1
3  88   21  108    0   0   1    2  2594           0           0             1
4  89   18  107    0   0   1    0  2600           0           0             1


UnboundLocalError: local variable 'ln_likelyhood' referenced before assignment

In [ ]:
data.head()

尤度関数の最適化をscipyの関数を使って行い、結果を出力  
引数として、group(stataのi.と同じでダミー変数を作る項を指定、defaultだと数値はそのまま扱い、文字列はダミー変数処理とかでいいと思う)  
ダミー変数で基準となる項目の指定とかも引数にできたらいいかも

おそらく一番数が多いやつがdefaultになっている。

In [11]:
data.dtypes

id          int16
low          int8
age          int8
lwt         int16
race     category
smoke    category
ptl          int8
ht           int8
ui           int8
ftv          int8
bwt         int16
dtype: object

In [12]:
category_variables = data.dtypes[(data.dtypes == 'category') | (data.dtypes == 'object')].index
category_mode = {}
for variable in category_variables:
    category_mode[variable] = data[variable].value_counts().index[0]
print(category_mode)

{'race': 'white', 'smoke': 'nonsmoker'}


In [13]:
def get_category_modes(df):
    """
    カテゴリータイプまたはオブジェクトタイプの列の最頻値を返す関数
    """
    category_variables = df.dtypes[(df.dtypes == 'category') | (df.dtypes == 'object')].index
    category_mode = {}
    for variable in category_variables:
        category_mode[variable] = df[variable].value_counts().index[0]
    return category_mode

In [14]:
X_dummy = pd.get_dummies(X)
category_modes = get_category_modes(X)
for index, value in category_modes.items():
    X_dummy.drop(index + '_' + value, axis=1, inplace=True)

In [15]:
len(X_dummy.iloc[0])

11

In [16]:
X_dummy.iloc[0]

id                85
age               19
lwt              182
ptl                0
ht                 0
ui                 1
ftv                0
bwt             2523
race_black         1
race_other         0
smoke_smoker       0
Name: 0, dtype: int16

In [17]:
X_dummy.iloc[0]*np.array([3, 4, 0, 1, 2, 2, 2, 2, 2, 2, 2])

id               255
age               76
lwt                0
ptl                0
ht                 0
ui                 2
ftv                0
bwt             5046
race_black         2
race_other         0
smoke_smoker       0
Name: 0, dtype: int64

In [18]:
np.dot(X_dummy.iloc[0], np.array([3, 4, 0, 1, 2, 2, 2, 2, 2, 2, 2]))

5381

In [125]:
def logistic_ln_likelyhood(b, X, y):
    """
    bはdummy化したXの列と同じ長さのnp.arrayを想定
    """
    ln_likelyhood = 0
    for X_i, y_i in zip(X.itertuples(name=None), y):
        if y_i:
            ln_likelyhood += math.log(math.exp(np.dot(X_i, b))/(1+math.exp(np.dot(X_i, b))))
        else:
            ln_likelyhood += 1-math.log(math.exp(np.dot(X_i, b))/(1+math.exp(np.dot(X_i, b))))
    print(ln_likelyhood)
    return - ln_likelyhood

def ln_likelyhood_func(X, y):
    return functools.partial(logistic_ln_likelyhood, X=X, y=y)

In [126]:
import functools
import math
from scipy import optimize

In [127]:
X_dummy_ = X_dummy.copy()
bwt = X_dummy['bwt']
X_dummy_['bwt'] = (bwt-bwt.mean()) / bwt.std()
lwt = X_dummy['lwt']
X_dummy_['lwt'] = (lwt-lwt.mean()) / lwt.std()
age = X_dummy['age']
X_dummy_['age'] = (age-age.mean()) / age.std()

In [128]:
func = ln_likelyhood_func(X_dummy_.drop('id', axis=1).drop('bwt', axis=1).drop('ftv', axis=1), y)

In [135]:
b = np.array([0.1, 3, 3, 3, 1, 3, 1, 6, 3])
func(b)

145.91816344796706


-145.91816344796706

In [136]:
optimize.minimize(func, b)

145.91816344796706
145.9181617301531
145.91816355074027
145.91816352839223
145.9181634479134
145.91816344752732
145.91816344286661
145.9181634086798
145.91816343845784
145.918163403816
145.91816344796706
-1039.7225521469738
-1039.7225521469738
-1039.7225370455092
-1039.7225524283476
-1039.7225525670913
-1039.7225520894156
-1039.7225521171679
-1039.7225521118455
-1039.7225521916955
-1039.722552366384
-1039.722552345089
148.7896878128931
148.7896878128931
148.789685526444
148.7896879300247
148.78968790230994
148.7896878127882
148.78968781202016
148.7896878062583
148.78968776735687
148.7896878018694
148.78968776369587
154.0580766263043
154.0580766263043
154.05807294991487
154.05807677014775
154.0580767302857
154.05807662606534
154.05807662396427
154.05807661550426
154.05807657119632
154.05807661222167
154.0580765673585
-539.9608376638016
-539.9608376638016
-539.9608210403567
-539.9608378696893
-539.9608380481142
-539.9608375997223
-539.9608376326223
-539.9608376121141
-539.9608377030454
-

OverflowError: math range error

In [86]:
X_dummy_.head().drop('id', axis=1)

,age,lwt,ptl,ht,ui,ftv,bwt,race_black,race_other,smoke_smoker
0,-0.799840,1.706611,0,0,1,0,-0.577883,1,0,0
1,1.842328,0.823541,0,0,0,3,-0.539475,0,1,0
2,-0.611114,-0.811774,0,0,0,1,-0.531244,0,0,1
3,-0.422387,-0.713655,0,0,1,2,-0.480491,0,0,1
4,-0.988566,-0.746361,0,0,1,0,-0.472261,0,0,1
